In [ ]:
# load the libraries
library(tidycensus)
library(tidyr)
library(stringr)
library(dplyr)


# load your census api key with an environment variable
census_api_key(Sys.getenv("CENSUS_API_KEY"))


# load variable data and transform
variable_data <- load_variables(2020, "acs5", cache = TRUE) %>%
  rename_all(recode,
    name = "variable_key", concept = "dataset",
    label = "variable"
  ) %>%
  mutate(
    dataset = tolower(dataset),
    dataset = gsub(" ", "_", dataset),
    variable = tolower(variable),
    variable = gsub("!!", "_", variable),
    variable = gsub(" ", "_", variable),
    variable = gsub(":", "", variable)
  ) %>%
  select(-geography)


# list of counties
counties <- c(
  "adams", "asotin", "ferry", "garfield", "lincoln",
  "pend oreille", "spokane", "stevens", "whitman"
)


# load census data and transform
getCensusData <- function(table) {
  census_data <- get_acs(
    geography = "tract",
    table = table,
    year = 2020,
    state = "WA",
    survey = "acs5",
    cache_table = FALSE
  ) %>%
    separate(NAME, c("tract", "county", "state"), sep = ",") %>%
    mutate(
      tract = gsub("Census Tract ", "", tract),
      tract = as.double(tract),
      county = tolower(county),
      county = gsub(" county", "", county),
      county = gsub(" ", "", county)
    ) %>%
    rename_all(recode, variable = "variable_key") %>%
    select(-state, -GEOID, variable_key) %>%
    filter(county %in% counties)

  return(census_data)
}


# load urban data and transform
getUrbanCensusData <- function(table) {
  census_data <- get_acs(
    geography = "urban area",
    table = table,
    year = 2020,
    survey = "acs5",
    cache_table = FALSE
  ) %>%
    filter(str_detect(NAME, "Spokane")) %>%
    rename_all(recode, variable = "variable_key") %>%
    merge(variable_data, by = "variable_key") %>%
    select(-GEOID, -NAME, -variable_key)


  return(census_data)
}

naturalization <- getCensusData("B05011")
naturalization_urban <- getUrbanCensusData("B05011")


nativity <- getCensusData("B05012")
nativity_urban <- getUrbanCensusData("B05012")


transportation <- getCensusData("B08101")
transportation_urban <- getUrbanCensusData("B08101")


type_computer <- getCensusData("B28001")
type_computer_urban <- getUrbanCensusData("B28001")


presence_computer <- getCensusData("B28003")
presence_computer_urban <- getUrbanCensusData("B28003")


internet_subscription <- getCensusData("B28011")
internet_subscription_urban <- getUrbanCensusData("B28011")


voting_age <- getCensusData("B29002")
voting_age_urban <- getUrbanCensusData("B29002")


household_income <- getCensusData("S1901")
household_income_urban <- getUrbanCensusData("S1901")



combined_data <- rbind(
  naturalization,
  nativity,
  transportation,
  type_computer,
  presence_computer,
  internet_subscription,
  voting_age,
  household_income
)


combined_data_urban <- rbind(
  naturalization_urban,
  nativity_urban,
  transportation_urban,
  type_computer_urban,
  presence_computer_urban,
  internet_subscription_urban,
  voting_age_urban,
  household_income_urban
)


dirty_data <- get_acs(
  geography = "tract",
  table = "B28003",
  year = 2020,
  state = "WA",
  survey = "acs5",
  cache_table = FALSE
)

clean_data <- getCensusData("B28003")
